# Project: Image/Object Detection Portfolio (Optional)

**Instructions for Students:**

Please carefully follow these steps to complete and submit your assignment:

1. **Completing the Assignment**: You are required to work on and complete all tasks in the provided assignment. Be disciplined and ensure that you thoroughly engage with each task.
   
2. **Creating a Google Drive Folder**: If you don't previously have a folder for collecting assignments, you must create a new folder in your Google Drive. This will be a repository for all your completed assignment files, helping you keep your work organized and easy to access.
   
3. **Uploading Completed Assignment**: Upon completion of your assignment, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
   
4. **Sharing Folder Link**: You're required to share the link to your assignment Google Drive folder. This is crucial for the submission and evaluation of your assignment.
   
5. **Setting Permission toPublic**: Please make sure your **Google Drive folder is set to public**. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth assignment process for you and the reviewers.

**Description:**

Welcome to your image/object detection portfolio project assignment for AI Bootcamp. In this project you will apply what you've learned so far into real-world applications.

You are free to create anything as long as it's within the category of image based or object detection application or model.

Some ideas for your applications:
* Image Classification application that can correctly identify local Indonesian food.
* Object Detection application that can identify correctly animals inside the picture.
* Object Detection application that can identify correctly animals inside a video.

For submission, please upload the model and application to Huggingface or your own Github account.

Your submission will be graded and scored and will add a maximum of 15 points to your final score.

Remember, the key to mastering these concepts is practice. Do apply your knowledge, and don't hesitate to ask questions if you encounter any difficulties. Good luck!

In [1]:
# @title #### Student Identity
student_id = "REA7YDK9" # @param {type:"string"}
name = "Mangara Haposan Immanuel Siagian" # @param {type:"string"}
drive_link = "https://drive.google.com/drive/folders/1oRRHNoCZQ5h7qiUKiEkQ0zGIBP27NvUI?usp=sharing"  # @param {type:"string"}
assignment_id = "00_image_portfolio_project"

## Installation and Import `rggrader` Package

In [ ]:
%pip install rggrader
from rggrader import submit_image
from rggrader import submit

# Working Space

In [3]:
# Write your code here
from google.colab import files
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split

In [4]:
!pip install -q kaggle

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d alessiocorrado99/animals10
!unzip animals10.zip

In [7]:
data_dir = '/content/raw-img'

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(128, padding=10),
    transforms.ToTensor(),
])

dataset = ImageFolder(root=data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(dataset.classes)
print("Number of classes:", num_classes)

Number of classes: 10


In [8]:
class Animal(nn.Module):
    def __init__(self, num_classes=10):
        super(Animal, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.relu4(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = Animal(num_classes=10)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss / len(train_loader)}")

print("Training complete.")

Epoch 1/50 - Loss: 2.355769860835476
Epoch 2/50 - Loss: 1.9317672170755518
Epoch 3/50 - Loss: 1.8346729839121112
Epoch 4/50 - Loss: 1.7638332659961613
Epoch 5/50 - Loss: 1.6772121760681384
Epoch 6/50 - Loss: 1.6087647084971421
Epoch 7/50 - Loss: 1.5636871415240163
Epoch 8/50 - Loss: 1.5132019201307807
Epoch 9/50 - Loss: 1.474518609410934
Epoch 10/50 - Loss: 1.4167088011748918
Epoch 11/50 - Loss: 1.380776459661149
Epoch 12/50 - Loss: 1.3475199597482463
Epoch 13/50 - Loss: 1.324586610029672
Epoch 14/50 - Loss: 1.2906960429126069
Epoch 15/50 - Loss: 1.2656289619343881
Epoch 16/50 - Loss: 1.2393328466488205
Epoch 17/50 - Loss: 1.2174797716941543
Epoch 18/50 - Loss: 1.1897978723504161
Epoch 19/50 - Loss: 1.18845326090587
Epoch 20/50 - Loss: 1.1580153109463116
Epoch 21/50 - Loss: 1.1428411984261666
Epoch 22/50 - Loss: 1.1071994488475887
Epoch 23/50 - Loss: 1.0865366315113678
Epoch 24/50 - Loss: 1.0745721316519585
Epoch 25/50 - Loss: 1.0504196204301965
Epoch 26/50 - Loss: 1.0338013891045374
E

In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 74.62%


In [12]:
torch.save(model.state_dict(), '/content/model.pth')

In [ ]:
!pip install gradio
import gradio as gr
from PIL import Image

In [14]:
class_labels = ['Dog', 'Horse', 'Elephant', 'Butterfly', 'Chicken', 'Cat', 'Cow', 'Sheep', 'Spider', 'Squirrel']

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

model = Animal()
model.load_state_dict(torch.load('/content/model.pth'))
model.to('cuda')
model.eval()

def predict_class_with_confidence(input_image):
    input_image = Image.fromarray(input_image)
    input_image = transform(input_image).unsqueeze(0).to('cuda')

    with torch.no_grad():
        output = model(input_image)

    _, predicted_class = torch.max(output.data, 1)
    predicted_label = class_labels[predicted_class.item()]

    return predicted_label

iface = gr.Interface(
    fn=predict_class_with_confidence,
    inputs=gr.inputs.Image(),
    outputs="text",
    live=True,
    capture_session=True,
    title="Animal Classification App",
    description="Upload an image of an animal to classify it",
)

iface.launch()

<ipython-input-14-8d1e24576838>:27: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(),
<ipython-input-14-8d1e24576838>:27: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(),
<ipython-input-14-8d1e24576838>:25: GradioDeprecationWarning: `capture_session` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ffb73d361c51ca0827.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Submission

In [15]:
portfolio_link = "https://huggingface.co/spaces/Mangara01/Animal-Classification"

question_id = "01_image_portfolio_link"
submit(student_id, name, assignment_id, str(portfolio_link), question_id, drive_link)

'Assignment successfully submitted'

# FIN